<a href="https://colab.research.google.com/github/diwakarojha/Para-Phrasing/blob/main/ActiveVoiceToPassiveVoice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
!pip install pytorch_lightning

     |████████████████████████████████| 839kB 19.1MB/s 
     |████████████████████████████████| 829kB 50.9MB/s 
     |████████████████████████████████| 184kB 55.2MB/s 
     |████████████████████████████████| 276kB 51.7MB/s 
     |████████████████████████████████| 112kB 55.8MB/s 
     |████████████████████████████████| 1.3MB 45.6MB/s 
     |████████████████████████████████| 143kB 56.7MB/s 
     |████████████████████████████████| 296kB 51.3MB/s 
  Created wheel for future: filename=future-0.18.2-cp37-none-any.whl size=491058 sha256=5beb89933eeedb514fdf91061f40c59a31073a1f75c7618269fd716ea54d1f2f
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp37-cp37m-linux_x86_64.whl size=44620 sha256=85b561487486b6f1841923c473013744588a64c75d52e57d65382ce26477c208
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
Successfully built futur

In [3]:
!pip install transformers

     |████████████████████████████████| 2.0MB 17.7MB/s 
     |████████████████████████████████| 870kB 48.6MB/s 
     |████████████████████████████████| 3.2MB 54.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=dca55ef107bdc2cfede938d0d201175cb35196a36738d9cfd62d6c59598cfbd9
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses


In [4]:
!pip install sentencepiece

     |████████████████████████████████| 1.2MB 19.0MB/s 


In [5]:
import argparse
import glob
import os
import json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl

from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [6]:
def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)

set_seed(42)

In [7]:
class T5FineTuner(pl.LightningModule):
    def __init__(self, hparams):
        super(T5FineTuner, self).__init__()
        self.hparams = hparams

        self.model = T5ForConditionalGeneration.from_pretrained(hparams.model_name_or_path)
        self.tokenizer = T5Tokenizer.from_pretrained(hparams.tokenizer_name_or_path)

    def is_logger(self):
        return True #self.trainer.proc_rank <= 0

    def forward(
            self, input_ids, attention_mask=None, decoder_input_ids=None, decoder_attention_mask=None, lm_labels=None
    ):
        return self.model(
            input_ids,
            attention_mask=attention_mask,
            decoder_input_ids=decoder_input_ids,
            decoder_attention_mask=decoder_attention_mask,
            lm_labels=lm_labels,
        )

    def _step(self, batch):
        lm_labels = batch["target_ids"]
        lm_labels[lm_labels[:, :] == self.tokenizer.pad_token_id] = -100

        outputs = self(
            input_ids=batch["source_ids"],
            attention_mask=batch["source_mask"],
            lm_labels=lm_labels,
            decoder_attention_mask=batch['target_mask']
        )

        loss = outputs[0]

        return loss

    def training_step(self, batch, batch_idx):
        loss = self._step(batch)

        tensorboard_logs = {"train_loss": loss}
        return {"loss": loss, "log": tensorboard_logs}

    def training_epoch_end(self, outputs):
        avg_train_loss = torch.stack([x["loss"] for x in outputs]).mean()
        tensorboard_logs = {"avg_train_loss": avg_train_loss}
        return {"avg_train_loss": avg_train_loss, "log": tensorboard_logs, 'progress_bar': tensorboard_logs}

    def validation_step(self, batch, batch_idx):
        loss = self._step(batch)
        return {"val_loss": loss}

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
        tensorboard_logs = {"val_loss": avg_loss}
        return {"avg_val_loss": avg_loss, "log": tensorboard_logs, 'progress_bar': tensorboard_logs}

    def configure_optimizers(self):
        "Prepare optimizer and schedule (linear warmup and decay)"

        model = self.model
        no_decay = ["bias", "LayerNorm.weight"]
        optimizer_grouped_parameters = [
            {
                "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
                "weight_decay": self.hparams.weight_decay,
            },
            {
                "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
                "weight_decay": 0.0,
            },
        ]
        optimizer = AdamW(optimizer_grouped_parameters, lr=self.hparams.learning_rate, eps=self.hparams.adam_epsilon)
        self.opt = optimizer
        return [optimizer]

    def optimizer_step(self, epoch, batch_idx, optimizer, optimizer_idx, second_order_closure=None, on_tpu=False, using_native_amp=False, using_lbfgs=False):
        if self.trainer.use_tpu:
            xm.optimizer_step(optimizer)
        else:
            optimizer.step()
        optimizer.zero_grad()
        self.lr_scheduler.step()

    def get_tqdm_dict(self):
        tqdm_dict = {"loss": "{:.3f}".format(self.trainer.avg_loss), "lr": self.lr_scheduler.get_last_lr()[-1]}

        return tqdm_dict

    def train_dataloader(self):
        train_dataset = get_dataset(tokenizer=self.tokenizer, type_path="train", args=self.hparams)
        dataloader = DataLoader(train_dataset, batch_size=self.hparams.train_batch_size, drop_last=True, shuffle=True,
                                num_workers=4)
        t_total = (
                (len(dataloader.dataset) // (self.hparams.train_batch_size * max(1, self.hparams.n_gpu)))
                // self.hparams.gradient_accumulation_steps
                * float(self.hparams.num_train_epochs)
        )
        scheduler = get_linear_schedule_with_warmup(
            self.opt, num_warmup_steps=self.hparams.warmup_steps, num_training_steps=t_total
        )
        self.lr_scheduler = scheduler
        return dataloader

    def val_dataloader(self):
        val_dataset = get_dataset(tokenizer=self.tokenizer, type_path="dev", args=self.hparams)
        return DataLoader(val_dataset, batch_size=self.hparams.eval_batch_size, num_workers=4)

logger = logging.getLogger(__name__)

class LoggingCallback(pl.Callback):
  def on_validation_end(self, trainer, pl_module):
    logger.info("***** Validation results *****")
    if pl_module.is_logger():
      metrics = trainer.callback_metrics
      # Log results
      for key in sorted(metrics):
        if key not in ["log", "progress_bar"]:
          logger.info("{} = {}\n".format(key, str(metrics[key])))

  def on_test_end(self, trainer, pl_module):
    logger.info("***** Test results *****")

    if pl_module.is_logger():
      metrics = trainer.callback_metrics

      # Log and save results to file
      output_test_results_file = os.path.join(pl_module.hparams.output_dir, "test_results.txt")
      with open(output_test_results_file, "w") as writer:
        for key in sorted(metrics):
          if key not in ["log", "progress_bar"]:
            logger.info("{} = {}\n".format(key, str(metrics[key])))
            writer.write("{} = {}\n".format(key, str(metrics[key])))

In [8]:
# Load Dataset
data = pd.read_excel("/content/Active to Passive Dataset.xlsx",)#.astype(str)
data.tail(150)

,Active,Passive
0,Harry ate six shrimp at dinner.,"At dinner, six shrimp were eaten by Harry."
1,Beautiful giraffes roam the savannah.,The savannah is roamed by beautiful giraffes.
2,Sue changed the flat tire.,The flat tire was changed by Sue.
3,We are going to watch a movie tonight.,A movie is going to be watched by us tonight.
4,I ran the obstacle course in record time.,The obstacle course was run by me in record ti...
...,...,...
111,Drinking coffee is something that many profess...,Drinking coffee is recommended by many profess...
112,By drinking approximately 2.5 or 3 liters of w...,By drinking approximately 2.5 or 3 liters of w...
113,We’ve been calling her phone for a long time b...,Her phone is called by us for a long time but ...
114,"After talking to me, some things about my life...","After I was talked by you, some things have be..."


In [9]:
data_train=data.iloc[:100,:]
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Active   100 non-null    object
 1   Passive  100 non-null    object
dtypes: object(2)
memory usage: 1.7+ KB


In [10]:
data_dev=data.iloc[100:108,:]
data_dev.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 100 to 107
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Active   8 non-null      object
 1   Passive  8 non-null      object
dtypes: object(2)
memory usage: 260.0+ bytes


In [11]:
data_test=data.iloc[108:,:]
data_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 108 to 115
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Active   8 non-null      object
 1   Passive  8 non-null      object
dtypes: object(2)
memory usage: 260.0+ bytes


In [12]:
print('Training data: ', data_train.shape)
print('Validation data: ', data_dev.shape)
print('Test data: ', data_test.shape)

Training data:  (100, 2)
Validation data:  (8, 2)
Test data:  (8, 2)


In [13]:
!mkdir data

In [36]:
#data_train_1 = data_train_1.drop(['id', 'label'], axis=1).reset_index(drop=True)
#data_dev_1 = data_dev_1.drop(['id', 'label'], axis=1).reset_index(drop=True)
#data_test_1 = data_test_1.drop(['id', 'label'], axis=1).reset_index(drop=True)

In [37]:
#data_train_1.to_csv ("data/train.tsv", sep='\t',index = False)
#data_dev_1.to_csv ("data/dev.tsv", sep='\t',index = False)
#data_test_1.to_csv ("data/test.tsv", sep='\t',index = False)

In [14]:
data_train.to_csv("data/train.tsv",sep='\t',index = False)
data_dev.to_csv("data/dev.tsv",sep='\t', index = False)
data_test.to_csv("data/test.tsv",sep='\t', index = False)

## Set arguments

In [21]:
args_dict = dict(
    data_dir="", # path for data files
    output_dir="", # path to save the checkpoints
    model_name_or_path='t5-small',
    tokenizer_name_or_path='t5-small',
    max_seq_length=256,
    learning_rate=3e-4,
    weight_decay=0.0,
    adam_epsilon=1e-8,
    warmup_steps=0,
    train_batch_size=1,
    eval_batch_size=1,
    num_train_epochs=2,
    gradient_accumulation_steps=1,
    n_gpu=1,
    early_stop_callback=False,
    fp_16=False, # if you want to enable 16-bit training then install apex and set this to true
    opt_level='O1', # you can find out more on optimisation levels here https://nvidia.github.io/apex/amp.html#opt-levels-and-properties
    max_grad_norm=1.0, # if you enable 16-bit training then set this to a sensible value, 0.5 is a good default
    seed=42,
)

train_path = "data/train.tsv"
val_path = "data/dev.tsv"

train = pd.read_csv(train_path, sep="\t").astype(str)
print(train.head())

tokenizer = T5Tokenizer.from_pretrained('t5-small')

                                       Active                                            Passive
0            Harry ate six shrimp at dinner.         At dinner, six shrimp were eaten by Harry. 
1      Beautiful giraffes roam the savannah.      The savannah is roamed by beautiful giraffes. 
2                 Sue changed the flat tire.                  The flat tire was changed by Sue. 
3     We are going to watch a movie tonight.      A movie is going to be watched by us tonight. 
4  I ran the obstacle course in record time.   The obstacle course was run by me in record ti...


In [22]:
print(tokenizer)

PreTrainedTokenizer(name_or_path='t5-small', vocab_size=32100, model_max_len=512, is_fast=False, padding_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<extra_id_42>', '<extra_id_43>', '<extra_id_44>', '<extra_id_45>'

In [61]:
# from transformers import AutoTokenizer
# from transformers import AutoModelForSeq2SeqLM

# tokenizer = AutoTokenizer.from_pretrained("patrickvonplaten/t5-tiny-random")

# model = AutoModelForSeq2SeqLM.from_pretrained("patrickvonplaten/t5-tiny-random")

NameError: ignored

## ParaphraseDataset()

In [23]:
class ParaphraseDataset(Dataset):
    def __init__(self, tokenizer, data_dir, type_path, max_len=512):
        self.path = os.path.join(data_dir, type_path + '.tsv')

        self.source_column = "Active"
        self.target_column = "Passive"
        self.data = pd.read_csv(self.path, sep="\t").astype(str)

        self.max_len = max_len
        self.tokenizer = tokenizer
        self.inputs = []
        self.targets = []

        self._build()

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, index):
        source_ids = self.inputs[index]["input_ids"].squeeze()
        target_ids = self.targets[index]["input_ids"].squeeze()

        src_mask = self.inputs[index]["attention_mask"].squeeze()  # might need to squeeze
        target_mask = self.targets[index]["attention_mask"].squeeze()  # might need to squeeze

        return {"source_ids": source_ids, "source_mask": src_mask, "target_ids": target_ids, "target_mask": target_mask}

    def _build(self):
        for idx in range(len(self.data)):
            input_, target = self.data.loc[idx, self.source_column], self.data.loc[idx, self.target_column]

            input_ = "paraphrase: "+ input_ + ' </s>'
            target = target + " </s>"

            # tokenize inputs
            tokenized_inputs = self.tokenizer.batch_encode_plus(
                [input_], max_length=self.max_len, pad_to_max_length=True, return_tensors="pt", truncation='longest_first'
            )
            # tokenize targets
            tokenized_targets = self.tokenizer.batch_encode_plus(
                [target], max_length=self.max_len, pad_to_max_length=True, return_tensors="pt", truncation='longest_first'
            )

            self.inputs.append(tokenized_inputs)
            self.targets.append(tokenized_targets)

## Start Training

In [24]:
dataset = ParaphraseDataset(tokenizer, 'data', 'dev', 256)
print("Val dataset: ",len(dataset))

data = dataset[1]
print(tokenizer.decode(data['source_ids']))
print(tokenizer.decode(data['target_ids']))

if not os.path.exists('t5_paraphrase'):
    os.makedirs('t5_paraphrase')

args_dict.update({'data_dir': 'data', 'output_dir': 't5_paraphrase', 'num_train_epochs':10,'max_seq_length':256})
args = argparse.Namespace(**args_dict)
print(args_dict)

checkpoint_callback = pl.callbacks.ModelCheckpoint(
    dirpath=args.output_dir, prefix="checkpoint", monitor="val_loss", mode="min", save_top_k=5
)

train_params = dict(
    accumulate_grad_batches=args.gradient_accumulation_steps,
    gpus=args.n_gpu,
    max_epochs=args.num_train_epochs,
 #   early_stop_callback=False,
    precision= 16 if args.fp_16 else 32,
    amp_level=args.opt_level,
    gradient_clip_val=args.max_grad_norm,
    checkpoint_callback=checkpoint_callback,
    callbacks=[LoggingCallback()],
)

def get_dataset(tokenizer, type_path, args):
  return ParaphraseDataset(tokenizer=tokenizer, data_dir=args.data_dir, type_path=type_path,  max_len=args.max_seq_length)

print ("Initialize model")
model = T5FineTuner(args)

trainer = pl.Trainer(**train_params)

print (" Training model")
trainer.fit(model)

print ("training finished")

print ("Saving model")
model.model.save_pretrained('t5_paraphrase')

print ("Model saved")

!cp "/content/t5_paraphrase/" -a "/content/drive/My Drive/"
!cp "/content/lightning_logs/" -a "/content/drive/My Drive/"
print ("Copied the final folder to Google Drive")

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2074: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:175: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."


Val dataset:  8
paraphrase: Our work was really successful, but we could not prevent a small error at the end of the work. It was very sad for everyone to experience this situation.</s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


 Training model



  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 60.5 M
-----------------------------------------------------
60.5 M    Trainable params
0         Non-trainable params
60.5 M    Total params
242.026   Total estimated model params size (MB)
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


TypeError: ignored

## Start Testing

In [ ]:
import torch
from transformers import T5ForConditionalGeneration,T5Tokenizer

def set_seed(seed):
  torch.manual_seed(seed)
#  if torch.cuda.is_available():
#   torch.cuda.manual_seed_all(seed)

set_seed(42)

best_model_path = "drive/My Drive/Inabia NLP Models/T5-large-fine-tuned-2 epoch (PAWS)/t5_paraphrase"
model = T5ForConditionalGeneration.from_pretrained(best_model_path)
tokenizer = T5Tokenizer.from_pretrained('t5-large')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print ("device ",device)
model = model.to(device)

In [ ]:
sentence_1 = "Washing your hands Properly will keep you away from COVID-19."
sentence_2 = "Wikipedia was launched on January 15, 2001, and was created by Jimmy Wales and Larry Sanger."
sentence_3 = "NLP is one of the interesting fields for Data Scientists to focus on."
sentence_4 = "Do I really need to take a flu shot if I’m healthy with few or no underlying conditions?"
sentence_5 = "Which course should I take to get started in data science?"
sentence_6 = "There will be 3 Walmart Black Friday events held in November starting on November 4, November 11 and November 25!"
sentence_7 = "The FCC says the $200 million civil penalty is the largest fixed-amount penalty in the commission's history."
sentence_8 = "Southwest Airlines travelers can now fly directly from San Diego to Honolulu on a new service that took off Wednesday out of the San Diego International Airport."
sentence_9 = "Gasoline production averaged 9.1 million bpd last week, slightly down on the previous week."
sentence_10 = "If you fall into the latter group, here’s how to replace Google’s new icons for Gmail, Calendar, and other apps with the older, arguably better versions on Android, iPhone, and Chrome."
sentence_11 = "Apple has been working on ARM-based Macs for some time, but only made them official at this year's WWDC."
sentence_12 = "Microsoft is investigating reports that some users are seeing error 0x80070426 when using their Microsoft account to sign into various apps."
sentence_13 = "On Saturday, Connery’s family announced that the Oscar-winning Scottish actor died peacefully in his sleep at home in the Bahamas."
sentence_14 = "Baby Shark Dance, from South Korean brand Pinkfong, officially surpassed the song by Luis Fonsi as the most viewed YouTube video of all time, having racked up 7.05 billion views to 7.04 billion."
sentence_15 = "The University of Washington has informed the NFL office that due to an increase in COVID-19 infection rate and indications of increased community spread in the local area, NFL personnel are no longer allowed to attend games at Husky Stadium."
sentence_16 = "The NBA's basketball-related income was down $1.5 billion last season, according to data provided to teams and obtained by ESPN."
sentence_17 = "Yesterday, the huge orbiting laboratory celebrated 20 years of continuous human occupation, a big milestone in humanity's push to extend its footprint into the final frontier."
sentence_18 = "A team of researchers led by Osaka University and National Taiwan University created a system of nanoscale silicon resonators that can act as logic gates for light pulses."
sentence_19 = "The research on 100 people shows that all had T-cell responses against a range of the coronavirus’s proteins, including the spike protein used as a marker in many vaccine studies, after half a year."
sentence_20 = "A group of researchers at MIT recently developed an artificial intelligence model that can detect asymptomatic COVID-19 cases by listening to subtle differences in coughs between healthy people and infected people."

sentence = sentence_3

text =  "paraphrase: " + sentence + " </s>"

max_len = 256

encoding = tokenizer.encode_plus(text,pad_to_max_length=True, return_tensors="pt")
input_ids, attention_masks = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)

# set top_k = 50 and set top_p = 0.95 and num_return_sequences = 3
beam_outputs = model.generate(
    input_ids=input_ids, attention_mask=attention_masks,
    do_sample=True,
    max_length=256,
    top_k=120,
    top_p=0.98,
    early_stopping=True,
    num_return_sequences=1
)

print ("\nOriginal sentence: ")
print (sentence)
print ("\n")
print ("Paraphrased sentences: ")
final_outputs =[]
for beam_output in beam_outputs:
    sent = tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
    if sent.lower() != sentence.lower() and sent not in final_outputs:
        final_outputs.append(sent)

for i, final_output in enumerate(final_outputs):
    print("{}: {}".format(i, final_output))

In [ ]:
import torch
import pandas as pd
from transformers import T5ForConditionalGeneration,T5Tokenizer, AutoTokenizer

In [ ]:
def set_seed(seed):
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

set_seed(42)

best_model_path = "drive/My Drive/Inabia NLP Models/T5-large-fine-tuned-2 epoch (PAWS)/t5_paraphrase"
model = T5ForConditionalGeneration.from_pretrained(best_model_path)
tokenizer = T5Tokenizer.from_pretrained('t5-large')

In [ ]:
def similarity(x):
    input_ids = tokenizer.encode(str(x), return_tensors='pt')
    outputs = model.generate(input_ids=input_ids)
    similarityd = tokenizer.decode(outputs[0])
    return similarityd